In [1]:
"""
Script para pasar imagenes RGB y DEPTH a una visualizacion mas natural
RGB -> Giro 90 grados
DEPTH -> Giro 90 grados + Gray scaled.
"""

import numpy as np
import cv2
from IPython.display import Image,display
import imageio.v3 as iio
import glob
import os
original_depth_dir  = "datasets/evaluacion/depth_images/npy/"
visualization_depth_dir  = "datasets/evaluacion/depth_images/vis/"
original_rgb_dir    = "datasets/evaluacion/rgb_uhd_images/"
depths  = sorted(glob.glob(original_depth_dir+"*.npy"))
rgbs    = sorted(glob.glob(original_rgb_dir+"*.jpg"))

In [2]:
def create_intensity(image):
    return np.array(256 * (image / np.max(image)),dtype=np.uint8)

def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = Image(data=ret)
    display(i)
    
def rotate_image(mat, angle):
    height, width = mat.shape[:2]
    image_center = (width/2, height/2)
    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat


In [4]:
for depth,rgb in zip(depths,rgbs):
    # rgb rotate and save in the same directory
    cv2.imwrite(rgb,rotate_image(cv2.imread(rgb),90))
    # load, rotate, create intensity or gray scaled
    depth_filename  = depth.split(os.sep)[-1][:-4]
    depth_image     =    create_intensity(rotate_image(np.load(depth,allow_pickle=True),90))
    iio.imwrite(visualization_depth_dir+depth_filename+".png",depth_image )
    
